### Types of object detections
##### Template Matching: Looking for exact copy of a smaller img in bigger img
##### Corner Detection: Looking for corner in img Harris Corner and Shi-Tomasi detection
##### Edge Detection: Looking for general edges in img
##### Grid Detection: Combine both concept to find grids
##### Contour Detection: Detect foreground from background
##### Feature matching: Extract defining key features for matching. No need to use exact same image for matching

##### Watershed Algorithm: Allow us to segment foreground and background
##### Facial and eye detection: Haar Cascades to detect face and eyes but no recognition which need deep learning


In [4]:
# Template matching

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Feature matching and homograph

In [5]:
img1 = cv2.imread("assets/cat.jpg")
img2 = cv2.imread("assets/rotated_cat1.jpg")

# Detect and extract features from the images using SIFT
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Match the features using FLANN (Fast Library for Approximate Nearest Neighbors)
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Extract the good matches using the Lowe's ratio test
good_matches = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good_matches.append(m)

if len(good_matches) > 4:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ])
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ])
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()
    h,w,d = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    R = np.linalg.inv(M[:3,:3])
    angle = np.arctan2(R[1,0], R[0,0]) * 180 / np.pi
    print("Rotation angle difference: {:.2f} degrees".format(angle))

    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

# Draw the matches on the images
img3 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, matchesMask=matchesMask,
                      flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("Find angle difference between 2 images via homograph", img3)
cv2.waitKey()
cv2.destroyAllWindows()

Rotation angle difference: 44.99 degrees
